# COVID-19 for prediction in Castilla la Mancha
Notebook realizado para el **Máster en Ingeniería Informática** de la Universidad de Castilla la Mancha, para la asignatura Desarrollo de Sistemas Inteligentes.<br/>
En dicho Notebook se realizará una serie de predicciones empleando ARIMA (*autoregressive integrated moving average*), enfocado a la investigación del **COVID-19**, causada por el virus **SARS-CoV-2**, sobre la población de **Castilla la Mancha**, y más específicamente sobre los hospitales de la región.<br/>
El motivo del análisis de la predicción en un ámbito más localizado ha sido elegido por los autores para la ayuda a la estimación de recursos tanto médicos como materiales para hacer frente a dicha enfermedad aportando las herramientas que nos aportan las nuevas tecnologías en la ayuda a la predicción de datos.<br/>
Realizado por:


*   Félix Ángel Martínez Muela
*   Miguel de la Cal Bravo



In [0]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

!pip install pmdarima
!pip install statsmodels --upgrade

Importamos a nuestro entorno de ejecución en Google Colab nuestro dataset, en este caso hospitalizados, aunque valdría cualquiera de los presentes en la carpeta data/ , aunque habría que modificar el nombre del archivo en la siguiente línea.

In [0]:
from google.colab import files
uploaded = files.upload()

Tratamiento from CSV to Data Frame

1.   Carga del fichero.
2.   Ajuste del nombre de las comunas.
3.   Conversión de los tipos.
4.   Relleno con 0 los valores anteriores al primer dato.
5.   Interpolación lineal con los datos que faltan entre días (fin de semana sin datos etc).

In [0]:
df = pd.read_csv('hospitalizados.csv',encoding = "ISO-8859-1", index_col=0, header=None).T  # Lectura del fichero de entrada y transpuesta del mismo
df = df.rename(columns={'Fecha': 'Date'}) # Ajuste del nombre de las columnas
df['Date']= pd.to_datetime(df['Date']) # Conversión a tipo fecha
df.set_index('Date', inplace=True) # La columna Fechas como índice
df=df.apply(pd.to_numeric) # Hacer que los tipos de datos sean numéricos
df = df.sort_values('Date') # Ordenar en función de la fecha
df=df.interpolate('zero', fill_value=0, limit_direction='backward') # A 0 los primeros valores antes de un número
df=df.interpolate(method='linear', axis=0).ffill().bfill() # Valores intermedios los interpolas
df

# Auto-ARIMA<br/>
Definición de auto_arima:
Como parámetros de entrada, se le pasa el **dataset** ya corregido y luego se puede indicar el **número de periódos a predecir**, o si no se indica se toma el valor por defecto de  30 días.<br/>
La función tiene unos parámetros que pueden ser modificados si queremos que la función baraje la posibilidad de otros valores para la p y la q.


In [0]:
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm

def auto_arima(df,n_periods=30):
  model = pm.auto_arima(df, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

  #print(model.summary())
  fc, confint = model.predict(n_periods=n_periods, return_conf_int=True)
  return fc,model,confint

**Plot ARIMA**<br/>
Definimos una función para la realización de plots de manera que mostraremos los valores hasta el día de hoy, y añadiremos en verde las predicciones futuras complementadas de manera sombreada los posibles valores dentro del intervalo de confianza.

In [0]:
!mkdir img/
def plot_arima(name,df,lower_series,upper_series):
  # Plot
  plt.plot(df)
  plt.plot(fc_series, color='darkgreen')
  plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

  plt.title("Forecasting of "+name)
  plt.ylim(df.min(), df.max())
  plt.xticks(rotation=90)  
  plt.savefig('img/'+name+'.jpg', bbox_inches='tight')
  plt.show()


**Save** ARIMA model and **Plot** it<br/>


In [0]:
!mkdir models/
def save_model_to_file(name,model):
  #Save model
  with open('models/model_'+name+'.txt', 'w') as f:
    print(model.summary(),file=f)

def plot_model(name,model):
  model.plot_diagnostics(figsize=(7,5))
  plt.savefig('models/model_'+name+'_plot.jpg', bbox_inches='tight')
  plt.show()

**Bucle principal**, donde iteramos sobre las columnas del dataframe y por tanto de los hospitales de Castilla la Mancha:
1.   Extraemos el dataframe relativo a un hospital.
2.   Llamamos a ARIMA, pasandole el hospital a analizar y el número de días vista a predecir.
3.   Creamos las series temporales con los datos de la predicción, con un índice de las fechas de las predicciones hasta ese día y de manera automática.
4.   Guardamos en un diccionario los modelos y predicciones, para futuras consultas.
5.   Creamos el plot de la prediccion llamando a plot_arima.
6.   Guardamos el modelo y sus plots relativos, llamando a save_model_to_file y a plot_model.
7.   Creamos un dataframe con todas las predicciones de todos los hospitales.
8.   Eliminamos del dataframe total todas aquellas predicciones negativas, ya que no es posible que haya hospitalizados negativos en este caso.



In [0]:
import datetime
models_dic= {}    
forecasting_dic={}
all_fc_series=[]
df_prediction = pd.DataFrame(data=None, columns=df.columns, index=df.index)

for column in df:
  #Split df in columns
  df_col = df.loc[:,column]
  #Auto Arima
  n_periods=30
  start_index=df.index[-1]+datetime.timedelta(days=1)
  fc_col, model_col, confint_col = auto_arima(df_col,n_periods)  

  #Series
  index_of_fc= pd.date_range(start=start_index, periods=n_periods, freq='D')
  fc_series = pd.Series(fc_col, index=index_of_fc)
  fc_series = fc_series.rename(column)
  all_fc_series.append(fc_series)
  lower_series = pd.Series(confint_col[:, 0], index=index_of_fc)
  upper_series = pd.Series(confint_col[:, 1], index=index_of_fc) 

  #Dictionaries
  models_dic[column]=model_col
  forecasting_dic[column]=fc_series

  #Plot
  plot_arima(column,df_col,lower_series,upper_series)

  #Saving model and plot it
  save_model_to_file(column,model_col)
  plot_model(column,model_col)
  
#Create a dataframe with all the forecastings
df_prediction = pd.DataFrame(all_fc_series).T

#Avoid negative predictions, because it is not possible in this dataset
df_prediction = df_prediction.clip(lower=0)


**Estado final** de las predicciones.<br/>
Como podemos observar se llama a info() para que nos indique la estructura del dataframe y también llamamos después al propio dataframe para ver los resultados finales.

In [0]:
df_prediction.info()
df_prediction

Unión de los datos actuales y las predicciones.

In [0]:
frames = [df, df_prediction]
df_new = pd.concat(frames)

In [0]:
a=df_new['H. Manzanares']
type(a[-1])

**Exportamos los datos** de las predicciones a excel (.csv)

*   Predicciones unico csv.
*   Predicciones por hospital.
*   Predicciones y datos de entrada.

De esta manera hemos conseguido tener ambos tipos de predicciones para poder compartirlas en función de las necesidades del destinatario, ya que si dichas predicciones se las queremos enviar a un hospital directamente, quizás no quieran tener más predicciones más hallá de las suyas.



In [0]:
!mkdir pred/
df_prediction.to_csv('pred/fc_series.csv', float_format='%.3f', decimal=",", sep = ';', encoding='ISO-8859-1')
for column in df:
  df_col = df_prediction.loc[:,column]
  df_col.to_csv('pred/fc_series_'+column+'.csv', float_format='%.3f', decimal=",", sep = ';', encoding='ISO-8859-1')
df_new.to_csv('pred/df_new.csv', float_format='%.3f', decimal=",", sep = ';', encoding='ISO-8859-1')

Descargamos los archivos a nuestro equipo<br/>
Si nos da un fallo, debemos irnos a la url de google chrome "chrome://settings/content/cookies" y añadir "colab.research.google.com"

In [0]:
!zip -r img.zip img
!zip -r models.zip models
!zip -r pred.zip pred
from google.colab import files
files.download("img.zip")
files.download("models.zip")
files.download("pred.zip")

Como conclusiones encontramos una estructura de directorios finales:


*   **img/**: tenemos las imágenes resultantes de las predicciones, empezando con el dataframe original, y continuando con las predicciones y sus intervalos de confianza.
*   **models/**: tenemos la descripción de los modelos y las gráficas de los modelos empleadas para cada predicción.
*   **pred/**: tenemos una predicción global y las predicciones individuales, todas en formato excel.

